**Test to see if all network components are working as expected**

In [1]:
from nebula3.gclient.net import ConnectionPool
from nebula3.Config import Config

In [2]:
class NebulaHandler:
    def __init__(self, space_name, host='nebula-docker-compose-graphd-1', port=9669, user='root', password='nebula'):
        """
        Initializes the NebulaHandler object and establishes a connection to NebulaGraph.

        Args:
            space_name (str): The name of the space to initialize and switch to.
            host (str): Host address of the NebulaGraph server.
            port (int): Port of the NebulaGraph server.
            user (str): Username for NebulaGraph.
            password (str): Password for NebulaGraph.
        """
        self.space_name = space_name
        self.config = Config()
        self.connection_pool = ConnectionPool()
        if not self.connection_pool.init([(host, port)], self.config):
            raise Exception("Failed to initialize the connection pool.")
        self.session = self.connection_pool.get_session(user, password)

In [3]:
handler = NebulaHandler(space_name='test_space', host='nebula-docker-compose-graphd-1', port=9669, user='root', password='nebula')

**https://docs.nebula-graph.io/3.8.0/3.ngql-guide/1.nGQL-overview/1.overview/**

In [4]:
handler.session.execute(f"""CREATE SPACE {handler.space_name}(partition_num=1, replica_factor=1, vid_type=INT64);""")
handler.session.execute(f"USE {handler.space_name};")

ResultSet(None)

In [5]:
result = handler.session.execute("SHOW SPACES;")
if result.is_succeeded():
    print("Spaces:", [row for row in result.rows()])
else:
    print("Error:", result.error_msg())


Spaces: [Row(
    values=[Value(
        sVal=b'test_space')])]


In [19]:
handler.session.execute("""CREATE TAG IF NOT EXISTS Person(name string, age int);""")
handler.session.execute("INSERT VERTEX Person(name, age) VALUES 3:('Charlie', 28);")
handler.session.execute("INSERT VERTEX Person(name, age) VALUES 4:('Amy', 22);")
handler.session.execute("INSERT VERTEX Person(name, age) VALUES 5:('Bob', 35);")
handler.session.execute("INSERT VERTEX Person(name, age) VALUES 6:('Alice', 27);")
handler.session.execute("INSERT VERTEX Person(name, age) VALUES 7:('David', 30);")
handler.session.execute("INSERT VERTEX Person(name, age) VALUES 8:('Eva', 19);")
handler.session.execute("INSERT VERTEX Person(name, age) VALUES 9:('Frank', 42);")

ResultSet(None)

**MATCH with id**

In [20]:
result = handler.session.execute("""MATCH (v:Person) WHERE id(v) == 3 RETURN v;""")
if result.is_succeeded():
    print("Data inserted successfully:", result.rows())
else:
    print("Failed to retrieve data:", result.error_msg())

Data inserted successfully: [Row(
    values=[Value(
        vVal=Vertex(
             vid=Value(
                 iVal=3),
             tags=[Tag(
                 name=b'Person',
                 props={b'age': Value(
                     iVal=28), b'name': Value(
                     sVal=b'Charlie')})]))])]


**MATCH property** - this needs creating an index for the property first!

In [23]:
handler.session.execute("""CREATE TAG INDEX idx_person_name ON Person(name(50));""")
handler.session.execute("""REBUILD TAG INDEX idx_person_name;""")
handler.session.execute("""SHOW TAG INDEXES;""")

ResultSet(keys: ['Index Name', 'By Tag', 'Columns'], values: ["idx_person_name", "Person", ["name"]])

In [24]:
resp = handler.session.execute("""MATCH (p:Person { name:'Charlie'})RETURN p;""")
if resp.is_succeeded():
    print("Retrieved data:", [row for row in resp.rows()])
else:
    print("Error:", resp.error_msg())


Retrieved data: [Row(
    values=[Value(
        vVal=Vertex(
             vid=Value(
                 iVal=3),
             tags=[Tag(
                 name=b'Person',
                 props={b'age': Value(
                     iVal=28), b'name': Value(
                     sVal=b'Charlie')})]))])]


Doing the same with WHERE

In [33]:
resp = handler.session.execute("""MATCH (p:Person) WHERE p.Person.name == 'Charlie' RETURN p;""")
if resp.is_succeeded():
    print("Retrieved data:", [row for row in resp.rows()])
else:
    print("Error:", resp.error_msg())


Retrieved data: [Row(
    values=[Value(
        vVal=Vertex(
             vid=Value(
                 iVal=3),
             tags=[Tag(
                 name=b'Person',
                 props={b'age': Value(
                     iVal=28), b'name': Value(
                     sVal=b'Charlie')})]))])]


**LOOKUP** with WHERE

In [30]:
resp = handler.session.execute("""LOOKUP ON Person WHERE Person.age > 30 YIELD Person.name;""")
if resp.is_succeeded():
    print("Retrieved data:", [row for row in resp.rows()])
else:
    print("Error:", resp.error_msg())


Retrieved data: [Row(
    values=[Value(
        sVal=b'Frank')]), Row(
    values=[Value(
        sVal=b'Bob')])]
